# Training Set Generation From Language Model

This notebook details how we can generate extra training data by using the first few words of a text as seed and generate variable-length sequences of a minority/often-misclassified class `pos` and `neg` to train the model.

In [45]:
import re

import emoji
import numpy as np
import pandas as pd

from pythainlp import word_tokenize
from tqdm import tqdm_notebook

# For visualization
import matplotlib.pyplot as plt
import seaborn as sns

from plotnine import *

# ulmfit
from fastai.text import *
from fastai.callbacks import CSVLogger, SaveModelCallback
from pythainlp.ulmfit import *

model_path = "wisesight_data/"


def replace_url(text):
    URL_PATTERN = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
    return re.sub(URL_PATTERN, 'xxurl', text)


def replace_rep(text):
    def _replace_rep(m):
        c, cc = m.groups()
        return f"{c}xxrep"
    re_rep = re.compile(r"(\S)(\1{2,})")

    return re_rep.sub(_replace_rep, text)


def ungroup_emoji(toks):
    res = []
    for tok in toks:
        if emoji.emoji_count(tok) == len(tok):
            for char in tok:
                res.append(char)
        else:
            res.append(tok)

    return res


def process_text(text):
    #pre rules
    res = text.lower().strip()
    res = replace_url(res)
    res = replace_rep(res)
    
    #tokenize
    res = [word for word in word_tokenize(res, engine="ulmfit") if word and not re.search(pattern=r"\s+", string=word)]
    
    #post rules
    res = ungroup_emoji(res)
    
    return res

In [46]:
all_df = pd.read_csv("all_df.csv")

all_df["processed"] = all_df.texts.map(lambda x: "|".join(process_text(x)))
all_df["wc"] = all_df.processed.map(lambda x: len(x.split("|")))
all_df["uwc"] = all_df.processed.map(lambda x: len(set(x.split("|"))))

In [47]:
# prevalence; let's generate like 2,000 of them
all_df.category.value_counts() 

neu    14243
neg     5713
pos     3917
q        472
Name: category, dtype: int64

In [48]:
seed_df = all_df[((all_df.category=="pos")|(all_df.category=="neg"))&((all_df.wc>=5)&(all_df.wc<=20))].reset_index(drop=True)
seed_df.wc.describe()

count    5191.000000
mean       10.681757
std         4.442878
min         5.000000
25%         7.000000
50%        10.000000
75%        14.000000
max        20.000000
Name: wc, dtype: float64

In [49]:
# lm data
data_lm = load_data(model_path, "wisesight_lm.pkl")
data_lm.sanity_check()

# learner
config = dict(emb_sz=400, n_hid=1550, n_layers=4, pad_token=1, qrnn=False, tie_weights=True, out_bias=True,
             output_p=0.25, hidden_p=0.1, input_p=0.2, embed_p=0.02, weight_p=0.15)
trn_args = dict(drop_mult=1., clip=0.12, alpha=2, beta=1)

learn = language_model_learner(data_lm, AWD_LSTM, config=config, pretrained=False, **trn_args)

#load pretrained models
learn.load("wisesight_lm");

In [50]:
# generate fake data
new_rows = []
for i,row in tqdm_notebook(seed_df.iterrows()):
    seed_words = "".join(row["processed"].split("|")[:3])
    n_words = np.random.randint(5,21)

    new_texts = learn.predict(seed_words, n_words=n_words, temperature=1.0, sep="")
    new_rows.append({"category": row["category"], "texts": new_texts, "processed": "", "wc": n_words, "uwc": 0})

fake_df = pd.DataFrame(new_rows)

In [51]:
# clean out tokens
fake_df["texts"] = fake_df.texts.map(lambda x: x.replace("xxbos", ""))
fake_df = fake_df[["category", "texts", "processed", "wc", "uwc"]]
fake_df.head(10)

,category,texts,processed,wc,uwc
0,pos,ก็น่าจะอร่อยกว่าบาบีก้อนอีกเลย,,5,0
1,neg,อ่านเม้นดิน.มีปัญหามีปัญหามีปัญหานะนะนะ คุณค...,,13,0
2,neg,โบกแท็กซี่สิบคันกินข้าวกับเพื่อนไม่ได้,,5,0
3,neg,ท่าทางยาบ้าเสียภาษีไปแล้ว 5 เปิดเปิดเปิด...,,12,0
4,neg,เสียภาษีก้อยอมรวมอยู่ไหนไม่ถึง # ลอรีเอะซูเปอร...,,15,0
5,neg,primerของrevlon 22องศา เพิ่มเติมค่ะ,,6,0
6,neg,"เบียร์ช้างขวดละ 10,0",,5,0
7,neg,??เอาไหม จัดไปได้ดี ไทก่อนเรา,,10,0
8,pos,"ไม่รุ้ว่าสอยเหนตัวไหนน้องซีใครบ้างยอม1,0ฟีฟีฟี...",,19,0
9,neg,ดูสถานะการณ์ที่ตรงกัน ว่าจะร้านอาหารแต่,,7,0


In [55]:
for i in range(20): print(i, fake_df.iloc[i,:].texts)

0 ก็น่าจะอร่อยกว่าบาบีก้อนอีกเลย
1 อ่านเม้นดิน.มีปัญหามีปัญหามีปัญหานะนะนะ   คุณคุณคุณทรายทรายทราย   สินสินสินชัยชัยชัยทองทองทอง   
2 โบกแท็กซี่สิบคันกินข้าวกับเพื่อนไม่ได้
3 ท่าทางยาบ้าเสียภาษีไปแล้ว 5       เปิดเปิดเปิดเปิดเปิดเปิดเปิดด่านด่านด่านด่านด่านด่านด่านทดสอบทดสอบทดสอบทดสอบทดสอบทดสอบทดสอบออนไลน์ออนไลน์ออนไลน์ออนไลน์ออนไลน์ออนไลน์ออนไลน์
4 เสียภาษีก้อยอมรวมอยู่ไหนไม่ถึง # ลอรีเอะซูเปอร์อัลตร้าสลิมสลิม
5 primerของrevlon 22องศา เพิ่มเติมค่ะ
6 เบียร์ช้างขวดละ 10,0
7 ??เอาไหม จัดไปได้ดี ไทก่อนเรา
8 ไม่รุ้ว่าสอยเหนตัวไหนน้องซีใครบ้างยอม1,0ฟีฟีฟีๆๆๆ444   เพื่อนเพื่อนเพื่อน
9 ดูสถานะการณ์ที่ตรงกัน ว่าจะร้านอาหารแต่
10 บะหมี่เป็ดเอ็มเค หมูสไลด์เป็นเมนูไทยมากค่ะ สาขา
11 บทเรียนนี้สอนให้รู้ว่าคนที่เล่นเกมส์นัดต่อไปใช้สิทธิ66 เพื่อไม่ให้เกิด
12 ให้ก็ไม่อยากได้ กินไม่เวอร์ เพราะเป็นตับ
13 ไปจ้าอาหื้มจะไปไหนดี106
14 สนไหมมากินt-bone อร่อยกว่า
15 ใครมาก็ได้ ลอรีเอะ ซูเปอร์ เจนเทิล เนื้อครีมครีม แสงคิ้ว 
16 เรโซน่าของออฟออฟยังไม่
17 รถไฟฟ้าวันทำงานวันไหนดีจิงๆ # เดินเก็บไว้ngansaisilp
18 ต้

In [52]:
all_aug = pd.read_csv("all_aug.csv")
fake_aug = pd.concat([all_aug,fake_df],0).reset_index(drop=True)
fake_aug.to_csv("fake_aug.csv", index=False)
fake_aug.shape

(33482, 5)

In [53]:
# prevalence
fake_aug.category.value_counts() 

neu    17039
neg     9782
pos     6158
q        503
Name: category, dtype: int64